In [ ]:
!pip install web3

In [ ]:
import time
import json
import requests
import pandas as pd
from tqdm import tqdm
from web3 import Web3
from web3.exceptions import ABIFunctionNotFound

In [ ]:
ETH_API_KEY = "NATDJD6I31EU3VE55UAD1FBY3VHGF8EE27"
ETH_URL = "https://api.etherscan.io/api"

RPC_URL = "https://rpc.ankr.com/eth/b1bdd2819d3da7be4760d270af7ef985a74725cbe337c348bca633d096806c5a"
w3 = Web3(Web3.HTTPProvider(RPC_URL))

getAccountBalance_url = "https://rpc.ankr.com/multichain/79258ce7f7ee046decc3b5292a24eb4bf7c910d7e39b691384c7ce0cfb839a01/?ankr_getAccountBalance="

In [ ]:
factories = {
    "sushiswap": {
        "address" : ["0xc0aee478e3658e2610c5f7a4a2e1777ce9e4f2ac"],
        "pool" : []
        }
}

In [ ]:
def get_abi(address):
    params = {
        "module": "contract",
        "action": "getabi",
        "address": address,
        "apikey": ETH_API_KEY
    }
    response = requests.post(ETH_URL, params=params)
    time.sleep(0.2)
    data = json.loads(response.text)
    result = data['result']
    return result

In [ ]:
def get_contract(address):
    abi = get_abi(address)
    checksum_address = Web3.to_checksum_address(address)
    contract = w3.eth.contract(address=checksum_address, abi=abi)
    return contract

In [ ]:
def get_fee_from_fee(contract, name):
    deno_dict = {
        "pancakeswap": 10000*100,
        "curve": 10**8,
        "uniswapv3": 10000*100
    }
    if name in deno_dict:
        fee = float(contract.functions.fee().call())
        fee = fee / deno_dict[name]
        return fee
    elif name == "fraxswap":
        fee = 10000 - float(contract.functions.fee().call())
        fee = fee / 10000
        return fee
    return None


# NomiswapStable
def get_fee_from_swapFee(contract, name):
    try:
        swapFee = float(contract.functions.swapFee().call())
        max_fee = float(contract.functions.MAX_FEE().call())
        fee = swapFee / max_fee
        return fee
    except Exception as e:
        return None


def get_fee_from_getSwapFeePercentage(contract, name):
    if name == "balancerv2":
        swapFeePercentage = float(contract.functions.getSwapFeePercentage().call())
        fee = swapFeePercentage / 10**18
        return fee
    return None


# saddle
def get_fee_from_swapStorage(contract, name):
    if name == "saddle":
        swapStorage = contract.functions.swapStorage().call()
        swapFee = float(swapStorage[4])
        fee = swapFee / 10**8
        return fee
    return None


def get_fee_from_tradingFeePercent(contract, name):
    if name == "linkswap":
        tradingFeePercent = float(contract.functions.tradingFeePercent().call())
        fee = tradingFeePercent / 10**6
        return fee
    return None


def get_fee_from_factory(contract, name):
    if name == "crodefiswap":
        factory_address = contract.functions.factory().call()
        factory_source = get_source_code(factory_address)
        factory_contract = get_contract(factory_address, factory_source)
        totalFeeBasisPoint = float(factory_contract.functions.totalFeeBasisPoint().call())
        fee = totalFeeBasisPoint / 10000
        return fee
    return None


def get_fee(contract, name):
    fix_fee_dict = {
        "uniswapv2": 3/1000,
        "sushiswap": 3/1000,
        "verseexchange": 3/1000,
        "saitaswap": 2/1000,
        "sakeswap_pool": 3/1000,
        "xchange": 200/100000,
        "kingswap": 25/10/1000,
    }

    fee_fun = [
        get_fee_from_fee,
        get_fee_from_swapFee,
        get_fee_from_getSwapFeePercentage,
        get_fee_from_swapStorage,
        get_fee_from_tradingFeePercent,
        get_fee_from_factory
    ]

    if name in fix_fee_dict:
        return fix_fee_dict[name]

    for fun in fee_fun:
        fee = fun(contract, name)
        if fee:
            break
    return fee

In [ ]:
def get_tokens(contract):
    try:
        token0 = contract.functions.token0().call().lower()
        token1 = contract.functions.token1().call().lower()
        return token0, token1
    except Exception:
        return None, None

In [ ]:
def get_pool_from_allPairs(factory, start, end):
    rows = []
    name = factory
    for factory_address in factories[factory]["address"]:
        factory_contract = get_contract(factory_address)
        try:
            allPairsLength = factory_contract.functions.allPairsLength().call()
            if end > allPairsLength:
                end = allPairsLength
            for i in tqdm(range(start, end)):
                try:
                    pool_address = factory_contract.functions.allPairs(i).call().lower()
                    contract = get_contract(pool_address)
                    fee = get_fee(contract, name)
                    token0, token1 = get_tokens(contract)
                    row = {
                        "name" : name,
                        "factory" : factory_address,
                        "fee" : fee,
                        "contract" : pool_address,
                        "token0" : token0,
                        "token1" : token1,
                    }
                    rows.append(row)
                    time.sleep(0.2)
                except ValueError:
                    continue
        except ABIFunctionNotFound:
            continue
    pool = pd.DataFrame(rows)
    pool = pool.drop_duplicates(ignore_index = True)
    return pool

In [ ]:
def get_tvl(pool):
    token_type = ['NATIVE']
    rows = []

    for i in tqdm(range(0, len(pool))):
        payload = {
            "jsonrpc": "2.0",
            "method": "ankr_getAccountBalance",
            "params": {
                "blockchain": ["eth"],
                "walletAddress": pool.contract[i],
                "nativeFirst": True,
                "onlyWhitelisted": True,
                "pageSize": 2
            },
            "id": 1
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json"
        }

        response = requests.post(getAccountBalance_url, json=payload, headers=headers)
        time.sleep(1)
        if response.status_code == 200:
            try:
                data = json.loads(response.text)
                tvl = data['result']['totalBalanceUsd']
            except KeyError:
                tvl = None
            row = {
                "name" : pool.name[i],
                "factory" : pool.factory[i],
                "tvl" : tvl,
                "fee" : pool.fee[i],
                "contract" : pool.contract[i],
                "token0" : pool.token0[i],
                "token1" : pool.token1[i],
            }

            rows.append(row)
    pool = pd.DataFrame(rows)
    pool = pool.dropna(axis=0).reset_index(drop=True)
    return pool

In [ ]:
for count in range(0, 10000):
    start = count * 1000
    end = start + 1000
    pool = get_pool_from_allPairs("sushiswap", start, end)
    if len(pool) <= 0:
        break
    pool = get_tvl(pool)
    file_name = f"part{count}.csv"
    pool.to_csv(file_name, index=False)

100%|█████████████████████████████████████████| 843/843 [28:13<00:00,  2.01s/it]
0it [00:00, ?it/s]


In [ ]:
files = [f"part{i}.csv" for i in range(count)]

In [ ]:
combined_df = pd.DataFrame()

In [ ]:
for file in files:
    df = pd.read_csv(file)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

combined_df = combined_df.drop_duplicates()
combined_df

,name,factory,tvl,fee,contract,token0,token1
0,sushiswap,0xc0aee478e3658e2610c5f7a4a2e1777ce9e4f2ac,1.093752e+03,0.003,0x680a025da7b1be2c204d7745e809919bce074026,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,0xdac17f958d2ee523a2206206994597c13d831ec7
1,sushiswap,0xc0aee478e3658e2610c5f7a4a2e1777ce9e4f2ac,1.768987e+07,0.003,0x06da0fd433c1a5d7a4faa01111c044910a184553,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xdac17f958d2ee523a2206206994597c13d831ec7
2,sushiswap,0xc0aee478e3658e2610c5f7a4a2e1777ce9e4f2ac,3.104047e+05,0.003,0x58dc5a51fe44589beb22e8ce67720b5bc5378009,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xd533a949740bb3306d119cc777fa900ba034cd52
3,sushiswap,0xc0aee478e3658e2610c5f7a4a2e1777ce9e4f2ac,1.309489e+03,0.003,0x95b54c8da12bb23f7a5f6e26c38d04acc6f81820,0xaba8cac6866b83ae4eec97dd07ed254282f6ad8a,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
4,sushiswap,0xc0aee478e3658e2610c5f7a4a2e1777ce9e4f2ac,4.692321e+04,0.003,0x611cde65dea90918c0078ac0400a72b0d25b9bb1,0x408e41876cccdc0f92210600ef50372656052a38,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
...,...,...,...,...,...,...,...
3838,sushiswap,0xc0aee478e3658e2610c5f7a4a2e1777ce9e4f2ac,8.352229e-02,0.003,0x670808f9eecd1f093d2698b3beaf6c67dce52bb5,0x42e92c12052ac80a4a2591d187c39931d0ca7097,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
3839,sushiswap,0xc0aee478e3658e2610c5f7a4a2e1777ce9e4f2ac,1.491261e-01,0.003,0x36a7c96b1bd5e99a3ad055be10a30e12a204f373,0x8cafc13eeba5cab3694aefc1631f3109a1c3bd1e,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
3840,sushiswap,0xc0aee478e3658e2610c5f7a4a2e1777ce9e4f2ac,2.451330e+02,0.003,0x80a7242e5c82199cdf219d210b14352804cb4c5f,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xca17243212e1969c06625efd45f4cdfb2647c5ef
3841,sushiswap,0xc0aee478e3658e2610c5f7a4a2e1777ce9e4f2ac,8.034075e+00,0.003,0xcd0c6aa05e07f15d8f74db3ef81b058a80456b1f,0x4db0c834e6e2966847760d68be694960a193c8c7,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2


In [ ]:
combined_df.to_csv('eth_sushiswap.csv', index=False)